# Makemytrip.com- Data Scraping

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import selenium.webdriver
from datetime import date
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from dateutil.relativedelta import relativedelta
import csv
import linecache
import sys

In [2]:
def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))

In [3]:
def scrap(origin,destin,trDate):
    global none
    baseDataUrl = "https://www.makemytrip.com/flight/search?itinerary="+ origin +"-"+ destin +"-"+ trDate +"&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E"

    try:
        driver = selenium.webdriver.Chrome('chromedriver.exe') 
        print ("Requesting URL: " + baseDataUrl)

        driver.get(baseDataUrl)  
        print ("Webpage found ...")

        element_xpath = '//*[@id="left-side--wrapper"]/div[2]'

        
        element = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.XPATH, element_xpath)))

        print ("Scrolling document upto bottom ...")
        for j in range(1, 100):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        
        body = driver.find_element_by_tag_name("body").get_attribute("innerHTML")

        print("Closing Chrome ...")
        driver.quit()

        print("Getting data from DOM ...")
        soupBody = BeautifulSoup(body)
        
       
        spanFlightName = soupBody.find_all("span", 'boldFont blackText airlineName')
        print("spanFlightName  = " + str(len(spanFlightName)))

        Time = soupBody.find_all("p", {"class":"blackText fontSize18 blackFont appendBottom2 makeFlex hrtlCenter"})			# Tags with Departure Time
        print("Time Count in Scraping  = " + str(len(Time)))

        City = soupBody.findAll("p", {"class": "darkText"})
        print("City Count in Scraping  = " + str(len(City)))

        spanFlightCost = soupBody.findAll("p", {"class": "blackText fontSize18 blackFont white-space-no-wrap"})			# Tags with Flight Cost
        print("spanFlightCost Count in Scraping  = " + str(len(spanFlightCost)))

        pFlightStops = soupBody.findAll("p", {"class": "flightsLayoverInfo"})
        print("pFlightStops Count in Scraping  = " + str(len(pFlightStops)))
        
        transform(spanFlightName,trDate,City,Time,spanFlightCost,pFlightStops)



   
    except :
        PrintException()
        print(trDate," scraping complete")
        
        none =True 

In [4]:
dates=(pd.date_range(start='15/10/2021', end='31/10/2021'))
New_date=[]
for i,date in enumerate(dates):
    day=str(date.day).zfill(2)   
    month=str(date.month).zfill(2)
    year=str(date.year)
    date=day+'/'+month+'/'+year  
    New_date.append(date)
print(New_date)

['15/10/2021', '16/10/2021', '17/10/2021', '18/10/2021', '19/10/2021', '20/10/2021', '21/10/2021', '22/10/2021', '23/10/2021', '24/10/2021', '25/10/2021', '26/10/2021', '27/10/2021', '28/10/2021', '29/10/2021', '30/10/2021', '31/10/2021']


In [5]:
#Lists to store scraped data
Airline=[]
Journey_date=[]
From=[]
To=[]
Price=[]
Dtime=[]
Atime=[]
Stops=[]

flight_data = pd.DataFrame(np.nan, index=[0], columns=['Airline', 'Journey_date','From','To','Dtime','Atime','Stops','Price'])
flight_data.to_csv('Flight.csv')
 
def transform(spanFlightName,trDate,City,Time,spanFlightCost,pFlightStops):
    
    Airline.clear()
    Journey_date.clear()
    Price.clear()
    Stops.clear()
    From.clear()
    Dtime.clear()
    To.clear()
    Atime.clear()
    
    global flight_data
    print("Transaform Function Running.... ")
    for i in range(0,len(spanFlightName)):
        
        Airline.append(spanFlightName[i].string)
        Journey_date.append(trDate)
        Price.append(spanFlightCost[i].string)
        Stops.append(pFlightStops[i].string)
        
      
    for j in range(0,len(Time)):
        
        if(j%2==0):
            
            From.append(City[j].string)
            Dtime.append(Time[j].string)
        else:
            
            To.append(City[j].string)
            Atime.append(Time[j].string) 

            
            
    temp = {'Airline':Airline,
            'Journey_date':Journey_date,
            'From':From,
            'To':To,
            'Dtime':Dtime,
            'Atime':Atime,
            'Stops':Stops,
            'Price':Price}
    df=pd.DataFrame(temp)
    df.to_csv('Flight.csv', mode='a', header=False)
    flight_data = pd.concat([flight_data,df])
    flight_data=flight_data.reset_index(drop=True)
    display(flight_data)
    print('Lenght : ',len(flight_data))

In [6]:
list_of_dest=['GOI','AMD','TIR','BLR','BOM','DEL','JAI','MAA','LKU','CCU','PNQ']
for i in range(0,len(list_of_dest)):
    orgin=list_of_dest[i]
    for_to=list_of_dest
    print("--------------")
    for j in range(0,len(for_to)):
        destin=for_to[j]
        none = False
        print("origin = "+orgin)
        print("destin = "+destin+'\n')
        if orgin!=destin:
            for i,date in enumerate(New_date) :
                if none !=True:
                    scrap(orgin,destin,date)
                    print(" ")
                else:
                    break
            else: 
                print( "-----Skiped-----")
                print(" ")

--------------
origin = GOI
destin = GOI

origin = GOI
destin = AMD

Requesting URL: https://www.makemytrip.com/flight/search?itinerary=GOI-AMD-15/10/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
Webpage found ...
Scrolling document upto bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 16
Time Count in Scraping  = 32
City Count in Scraping  = 32
spanFlightCost Count in Scraping  = 16
pFlightStops Count in Scraping  = 16
Transaform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Spicejet,15/10/2021,Goa,Ahmedabad,14:50,17:00,Non stop,"₹ 5,315"
2,Go First,15/10/2021,Goa,Ahmedabad,15:40,17:10,Non stop,"₹ 5,315"
3,Spicejet,15/10/2021,Goa,Ahmedabad,19:40,21:40,Non stop,"₹ 5,315"
4,Go First,15/10/2021,Goa,Ahmedabad,20:25,22:05,Non stop,"₹ 5,315"
5,Go First,15/10/2021,Goa,Ahmedabad,16:15,21:05,1 stop via Mumbai,"₹ 5,315"
6,Go First,15/10/2021,Goa,Ahmedabad,14:25,21:05,1 stop via Mumbai,"₹ 5,315"
7,Go First,15/10/2021,Goa,Ahmedabad,13:40,21:05,1 stop via Mumbai,"₹ 5,315"
8,Go First,15/10/2021,Goa,Ahmedabad,17:25,08:35,1 stop via New Delhi,"₹ 5,315"
9,IndiGo,15/10/2021,Goa,Ahmedabad,15:25,17:05,Non stop,"₹ 5,316"


Lenght :  17
 
Requesting URL: https://www.makemytrip.com/flight/search?itinerary=GOI-AMD-16/10/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
Webpage found ...
Scrolling document upto bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 26
Time Count in Scraping  = 52
City Count in Scraping  = 52
spanFlightCost Count in Scraping  = 26
pFlightStops Count in Scraping  = 26
Transaform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Spicejet,15/10/2021,Goa,Ahmedabad,14:50,17:00,Non stop,"₹ 5,315"
2,Go First,15/10/2021,Goa,Ahmedabad,15:40,17:10,Non stop,"₹ 5,315"
3,Spicejet,15/10/2021,Goa,Ahmedabad,19:40,21:40,Non stop,"₹ 5,315"
4,Go First,15/10/2021,Goa,Ahmedabad,20:25,22:05,Non stop,"₹ 5,315"
5,Go First,15/10/2021,Goa,Ahmedabad,16:15,21:05,1 stop via Mumbai,"₹ 5,315"
6,Go First,15/10/2021,Goa,Ahmedabad,14:25,21:05,1 stop via Mumbai,"₹ 5,315"
7,Go First,15/10/2021,Goa,Ahmedabad,13:40,21:05,1 stop via Mumbai,"₹ 5,315"
8,Go First,15/10/2021,Goa,Ahmedabad,17:25,08:35,1 stop via New Delhi,"₹ 5,315"
9,IndiGo,15/10/2021,Goa,Ahmedabad,15:25,17:05,Non stop,"₹ 5,316"


Lenght :  43
 
Requesting URL: https://www.makemytrip.com/flight/search?itinerary=GOI-AMD-17/10/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
Webpage found ...
Scrolling document upto bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 28
Time Count in Scraping  = 56
City Count in Scraping  = 58
spanFlightCost Count in Scraping  = 28
pFlightStops Count in Scraping  = 28
Transaform Function Running.... 


...

Lenght :  2236
 
Requesting URL: https://www.makemytrip.com/flight/search?itinerary=CCU-GOI-16/10/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
Webpage found ...
Scrolling document upto bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 31
Time Count in Scraping  = 62
City Count in Scraping  = 62
spanFlightCost Count in Scraping  = 31
pFlightStops Count in Scraping  = 31
Transaform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Spicejet,15/10/2021,Goa,Ahmedabad,14:50,17:00,Non stop,"₹ 5,315"
2,Go First,15/10/2021,Goa,Ahmedabad,15:40,17:10,Non stop,"₹ 5,315"
3,Spicejet,15/10/2021,Goa,Ahmedabad,19:40,21:40,Non stop,"₹ 5,315"
4,Go First,15/10/2021,Goa,Ahmedabad,20:25,22:05,Non stop,"₹ 5,315"
...,...,...,...,...,...,...,...,...
2262,"Spicejet, Go First",16/10/2021,Kolkata,Goa,08:50,15:10,1 stop via Bengaluru,"₹ 12,927"
2263,Spicejet,16/10/2021,Kolkata,Goa,09:00,11:20,Non stop,"₹ 12,998"
2264,Go First,16/10/2021,Kolkata,Goa,05:30,11:40,"2 stop via Jaipur,Ahmedabad","₹ 13,345"
2265,Vistara,16/10/2021,Kolkata,Goa,07:10,13:50,1 stop via New Delhi,"₹ 16,569"


Lenght :  2267
 
Requesting URL: https://www.makemytrip.com/flight/search?itinerary=CCU-GOI-17/10/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
Webpage found ...
Scrolling document upto bottom ...
Closing Chrome ...
Getting data from DOM ...
spanFlightName  = 29
Time Count in Scraping  = 58
City Count in Scraping  = 60
spanFlightCost Count in Scraping  = 29
pFlightStops Count in Scraping  = 29
Transaform Function Running.... 


,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Spicejet,15/10/2021,Goa,Ahmedabad,14:50,17:00,Non stop,"₹ 5,315"
2,Go First,15/10/2021,Goa,Ahmedabad,15:40,17:10,Non stop,"₹ 5,315"
3,Spicejet,15/10/2021,Goa,Ahmedabad,19:40,21:40,Non stop,"₹ 5,315"
4,Go First,15/10/2021,Goa,Ahmedabad,20:25,22:05,Non stop,"₹ 5,315"
...,...,...,...,...,...,...,...,...
2291,"IndiGo, Go First",17/10/2021,Kolkata,Goa,08:35,15:10,1 stop via Bengaluru,"₹ 13,871"
2292,"Spicejet, Go First",17/10/2021,Kolkata,Goa,08:50,15:10,1 stop via Bengaluru,"₹ 13,975"
2293,Go First,17/10/2021,Kolkata,Goa,05:30,11:40,1 stop via Jaipur,"₹ 15,640"
2294,Vistara,17/10/2021,Kolkata,Goa,07:10,13:50,1 stop via New Delhi,"₹ 17,934"


Lenght :  2296
 
Requesting URL: https://www.makemytrip.com/flight/search?itinerary=CCU-GOI-18/10/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
Webpage found ...
EXCEPTION IN (<ipython-input-3-d039f2a5c307>, LINE 15 "element = WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.XPATH, element_xpath)))"): Message: 

18/10/2021  scraping complete
 
origin = CCU
destin = AMD

Requesting URL: https://www.makemytrip.com/flight/search?itinerary=CCU-AMD-15/10/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
EXCEPTION IN (<ipython-input-3-d039f2a5c307>, LINE 9 "driver.get(baseDataUrl)"): Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=94.0.4606.61)

15/10/2021  scraping complete
 
origin = CCU
destin = TIR

Requesting URL: https://www.makemytrip.com/flight/search?itinerary=CCU-TIR-15/10/2021&tripType=O&paxType=A-1_C-0_I-0&intl=false&=&cabinClass=E
EXCEPTION IN (<ipython-input-3-d039f2a5c307>, LINE 9 "driver.get(baseDa

In [18]:
#checking scraped Flight Price data
df=pd.read_csv("Flight.csv")
df

,Unnamed: 0,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,Spicejet,15/10/2021,Goa,Ahmedabad,14:50,17:00,Non stop,"₹ 5,315"
2,1,Go First,15/10/2021,Goa,Ahmedabad,15:40,17:10,Non stop,"₹ 5,315"
3,2,Spicejet,15/10/2021,Goa,Ahmedabad,19:40,21:40,Non stop,"₹ 5,315"
4,3,Go First,15/10/2021,Goa,Ahmedabad,20:25,22:05,Non stop,"₹ 5,315"
...,...,...,...,...,...,...,...,...,...
2291,24,"IndiGo, Go First",17/10/2021,Kolkata,Goa,08:35,15:10,1 stop via Bengaluru,"₹ 13,871"
2292,25,"Spicejet, Go First",17/10/2021,Kolkata,Goa,08:50,15:10,1 stop via Bengaluru,"₹ 13,975"
2293,26,Go First,17/10/2021,Kolkata,Goa,05:30,11:40,1 stop via Jaipur,"₹ 15,640"
2294,27,Vistara,17/10/2021,Kolkata,Goa,07:10,13:50,1 stop via New Delhi,"₹ 17,934"


In [19]:
#Removing Columns "Unnamed:0 column as it will not affect our core dataset
df=df.drop('Unnamed: 0',axis=1,inplace=True)

In [22]:
#Removing First row as it have only NaN values
df=df.drop([df.index[0]])

In [23]:
#Equalizing Price column
df.Price = df.Price.str.replace('[^0-9.]','').astype('float64')
df

,Airline,Journey_date,From,To,Dtime,Atime,Stops,Price
1,Spicejet,15/10/2021,Goa,Ahmedabad,14:50,17:00,Non stop,5315.0
2,Go First,15/10/2021,Goa,Ahmedabad,15:40,17:10,Non stop,5315.0
3,Spicejet,15/10/2021,Goa,Ahmedabad,19:40,21:40,Non stop,5315.0
4,Go First,15/10/2021,Goa,Ahmedabad,20:25,22:05,Non stop,5315.0
5,Go First,15/10/2021,Goa,Ahmedabad,16:15,21:05,1 stop via Mumbai,5315.0
...,...,...,...,...,...,...,...,...
2291,"IndiGo, Go First",17/10/2021,Kolkata,Goa,08:35,15:10,1 stop via Bengaluru,13871.0
2292,"Spicejet, Go First",17/10/2021,Kolkata,Goa,08:50,15:10,1 stop via Bengaluru,13975.0
2293,Go First,17/10/2021,Kolkata,Goa,05:30,11:40,1 stop via Jaipur,15640.0
2294,Vistara,17/10/2021,Kolkata,Goa,07:10,13:50,1 stop via New Delhi,17934.0


In [24]:
df.dtypes

Airline          object
Journey_date     object
From             object
To               object
Dtime            object
Atime            object
Stops            object
Price           float64
dtype: object

In [25]:
#saving Flight Price data as csv file
df.to_csv("flight_price.csv")

In [26]:
#Shape of our Dataset after updation
print("Shape of our dataset is ",df.shape)
print("Column count in our dataset is",df.shape[1])
print("Row count in our dataset is ",df.shape[0])

Shape of our dataset is  (2295, 8)
Column count in our dataset is 8
Row count in our dataset is  2295


In [27]:
#Checking null values in Dataset
print("Empty cells in Dataset is ",df.isna().values.any())

Empty cells in Dataset is  False


# ||Happy Ending||